# Arabic Intent Classification using SetFit - Efficient Few-shot Learning with Sentence Transformers

Recently Huggingface and intel have published a paper on SetFit, a novel few-shot learning approach for sentence embeddings. SetFit is a simple and efficient method that can be used to train sentence embeddings for few-shot learning tasks. In this notebook, we will show how to use SetFit to train sentence embeddings for Arabic Intent Classification

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import plotly.express as px
from datasets import load_dataset
from sentence_transformers.losses import BatchAllTripletLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

## Loading of the Massive Intent Arabic Dataset from HuggingFace

In [ ]:
dataset = load_dataset("SetFit/amazon_massive_intent_ar-SA")
train_dataset = sample_dataset(dataset["train"], 128)
eval_dataset = dataset["test"]

## Setting up of Setfit Trainer and training of the model

In [ ]:
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=BatchAllTripletLoss,
    metric="accuracy",
    batch_size=64,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=16, # The number of epochs to use for contrastive learning
    warumup_proportion=0.2, # The proportion of the training steps to use for the warmup
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate()


In [ ]:

# Push model to the Hub
trainer.push_to_hub("fathyshalab/massive-ar-SA")

# Download from Hub and run inference
model = SetFitModel.from_pretrained("fathyshalab/massive-ar-SA")
# Run inference
preds = model(["تشغيل الأضواء", "كيف هو الطقس يوم الثلاثاء الثالث عشر"])